**Reading the Data**
* From csv file, and set the indexes

In [1]:
# import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import torch.optim as optim
import math

import matplotlib.pyplot as plt
torch.manual_seed(1)


Prepara data

In [2]:
class Net_1_layer_8_units(nn.Module):
    def __init__(self):
        super(Net_1_layer_8_units, self).__init__()
        
        self.fc1 = nn.Linear(25, 8)
        self.relu = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc3(x)
       
        return x

In [3]:
class Net_1_layer(nn.Module):
    def __init__(self):
        super(Net_1_layer, self).__init__()
        
        self.fc1 = nn.Linear(25, 4)
        self.relu = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc3(x)
       
        return x

In [4]:
class Net_2_layers(nn.Module):
    def __init__(self):
        super(Net_2_layers, self).__init__()
        
        self.fc1 = nn.Linear(25, 4)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
       
        return x

In [5]:
class Net_3_layers(nn.Module):
    def __init__(self):
        super(Net_3_layers, self).__init__()
        
        self.fc1 = nn.Linear(25, 4)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 4)
        self.fc4 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
       
        return x

In [6]:
def create_input_for_NN(X,y,num_hidden_layers = 1 ,mode='', lr =1e-2 ):
    
    if (num_hidden_layers == 1):
        net = Net_1_layer()
    if (num_hidden_layers == 2):
        net = Net_2_layers()
    if (num_hidden_layers == 3):
        net = Net_3_layers()
    else:
        net = Net_1_layer_8_units()

    pos_weight = torch.tensor(X[y.any_damage == 0].shape[0] / X[y.any_damage ==  1].shape[0 ], dtype=torch.float)
    #optimizer = optim.Adam(net.parameters(), lr=3e-3)
    #optimizer = optim.Adam(net.parameters(), lr=3e-2)
    optimizer = optim.Adam(net.parameters(), lr=1e-2)
    criterion = nn.BCEWithLogitsLoss()
    criterion_test = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


    X_tensor = torch.tensor(X.values, dtype=torch.float)
    y_tensor = torch.tensor(y.values, dtype=torch.float)

    assert X.shape[0] ==X_tensor.shape[0] , 'wrong input_df_vec dimensions'
    assert y.shape[0] ==y_tensor.shape[0] , 'wrong input_df_vec dimensions'
    assert len(X.columns) ==X_tensor.shape[1] , 'wrong input_df_vec dimensions'

    train_dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=30000, shuffle=True, num_workers=0)

    sm = SMOTE(random_state=0)
    X_os,y_os = sm.fit_resample(X, y)
    rus = RandomUnderSampler(random_state=0)
    X_us, y_us = rus.fit_resample(X, y)

    X_tensor_os = torch.tensor(X_os.values, dtype=torch.float)
    y_tensor_os = torch.tensor(y_os.values, dtype=torch.float)

    train_dataset_os = torch.utils.data.TensorDataset(X_tensor_os, y_tensor_os)
    train_loader_os = torch.utils.data.DataLoader(train_dataset_os, batch_size=30000, shuffle=True, num_workers=0)

    X_tensor_us = torch.tensor(X_us.values, dtype=torch.float)
    y_tensor_us = torch.tensor(y_us.values, dtype=torch.float)

    train_dataset_us = torch.utils.data.TensorDataset(X_tensor_us, y_tensor_us)
    train_loader_us = torch.utils.data.DataLoader(train_dataset_us, batch_size=30000, shuffle=True, num_workers=0)

    if (mode=='os'):
        return [net, optimizer, criterion,criterion_test,X_tensor_os, y_tensor_os,train_dataset_os,train_loader_os]
    elif (mode=='us'):
        return [net, optimizer, criterion,criterion_test,X_tensor_os, y_tensor_os,train_dataset_os,train_loader_os]
    elif (mode=='weight'):   
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        return [net, optimizer, criterion,criterion_test,X_tensor, y_tensor,train_dataset,train_loader]
    elif (mode=='unbalanced'):
        
        criterion_test = nn.BCEWithLogitsLoss()
        return [net, optimizer, criterion,criterion_test,X_tensor, y_tensor,train_dataset,train_loader]
    else:
        assert False, 'mode is incorrect'

In [7]:

def training_NN(net='', optimizer = '', criterion='',train_loader='', niter = 100):

    loss_vec=[]
    for epoch in range(niter):  # loop over the dataset multiple times

        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            
            loss = criterion(torch.flatten(outputs), torch.flatten(labels))
            loss.backward()
            optimizer.step()

            loss_vec.append(loss.detach().numpy())
    fig, axes = plt.subplots(1,1)
    ax = axes
    ax.set_xlabel('iterations')
    ax.set_ylabel('loss')
    ax.plot(loss_vec)

    return 
 